Pre-processing

In [2]:
!pip install "kagglehub[hf-datasets]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 51.3 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [datasets]/19 [datasets]ce-hub]


In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

file_path_db1 = "olist_order_items_dataset.csv"  

orderItems = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db1,
)

print("Hugging Face Dataset:", orderItems)

file_path_db2 = "olist_products_dataset.csv"  

items = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db2,
)

print("Hugging Face Dataset:", items)

file_path_db3 = "olist_orders_dataset.csv"  

order = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db3,
)

print("Hugging Face Dataset:", order)

orderItems = orderItems.to_pandas()
items = items.to_pandas()
order = order.to_pandas()

df = orderItems.merge(items, on="product_id", how="left") #merge 1
df = df.merge(order[["order_id", "order_purchase_timestamp"]], on="order_id", how="left")#merge2

# converter timestamp
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df.head()

df["month"] = df["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()

monthly = (
    df.groupby(["product_id", "month"])
      .agg(monthly_sales=("order_item_id", "count"))  
      .reset_index()
)

monthly = monthly.merge( #atributos estatícos
    items[[
        "product_id",
        "product_category_name",
        "product_name_lenght",
        "product_description_lenght",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    ]],
    on="product_id",
    how="left"
)

monthly.head() #assim temos as vendas por mês


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_4299/2328654540.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  orderItems = kagglehub.load_dataset(


100%|██████████| 6.37M/6.37M [00:00<00:00, 82.6MB/s]

Extracting zip of olist_order_items_dataset.csv...


Hugging Face Dataset: Dataset({
    features: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
    num_rows: 112650
})


/tmp/ipykernel_4299/2328654540.py:18: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  items = kagglehub.load_dataset(


...


/tmp/ipykernel_4299/2328654540.py:28: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  order = kagglehub.load_dataset(


100%|██████████| 6.75M/6.75M [00:00<00:00, 85.9MB/s]

Extracting zip of olist_orders_dataset.csv...


Hugging Face Dataset: Dataset({
    features: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'],
    num_rows: 99441
})


,product_id,month,monthly_sales,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00066f42aeeb9f3007548bb9d3f33c38,2018-05-01,1,perfumaria,53.0,596.0,6.0,300.0,20.0,16.0,16.0
1,00088930e925c41fd95ebfe695fd2655,2017-12-01,1,automotivo,56.0,752.0,4.0,1225.0,55.0,10.0,26.0
2,0009406fd7479715e4bef61dd91f2462,2017-12-01,1,cama_mesa_banho,50.0,266.0,2.0,300.0,45.0,15.0,35.0
3,000b8f95fcb9e0096488278317764d19,2018-08-01,2,utilidades_domesticas,25.0,364.0,3.0,550.0,19.0,24.0,12.0
4,000d9be29b5207b54e86aa1b1ac54872,2018-04-01,1,relogios_presentes,48.0,613.0,4.0,250.0,22.0,11.0,15.0


Defining features


In [ ]:
monthly = monthly.sort_values(["product_id", "month"])

for lag in [1, 2, 3]:
    monthly[f"lag_{lag}"] = (monthly.groupby("product_id")["monthly_sales"].shift(lag)
    )

monthly["rolling_3m"] = (
    monthly.groupby("product_id")["monthly_sales"].rolling(3).mean().reset_index(level=0, drop=True)
)

monthly["rolling_6m"] = (
    monthly.groupby("product_id")["monthly_sales"].rolling(6).mean().reset_index(level=0, drop=True)
)

monthly = monthly.dropna(subset=["lag_1", "lag_2", "lag_3"])

monthly.head()

,product_id,month,monthly_sales,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,lag_1,lag_2,lag_3,rolling_3m,rolling_6m
17,001b72dfd63e9833e8c02742adf472e3,2017-12-01,2,moveis_decoracao,45.0,520.0,3.0,600.0,26.0,8.0,22.0,1.0,2.0,2.0,1.666667,NaN
27,002159fe700ed3521f46cfcf6e941c76,2018-08-01,1,fashion_calcados,55.0,344.0,3.0,1850.0,36.0,37.0,16.0,1.0,2.0,1.0,1.333333,NaN
68,005030ef108f58b46b78116f754d8d38,2018-08-01,2,perfumaria,54.0,942.0,1.0,50.0,16.0,5.0,11.0,1.0,1.0,3.0,1.333333,NaN
102,008cff0e5792219fae03e570f980b330,2018-07-01,5,esporte_lazer,59.0,1735.0,5.0,1550.0,40.0,20.0,20.0,3.0,3.0,3.0,3.666667,NaN
103,008cff0e5792219fae03e570f980b330,2018-08-01,3,esporte_lazer,59.0,1735.0,5.0,1550.0,40.0,20.0,20.0,5.0,3.0,3.0,3.666667,NaN
